In [1]:
s = {1,2,3}

In [2]:
type(s)

<class 'set'>


In [3]:
import sys
config(channel is fifo, clock is lamport)

class P(process):
    def setup(s:set, nrequests:int):  # s is set of all other processes
        self.q = set()

    def mutex(task):
        -- request
        c = logical_clock()
        send(('request', c, self), to= s)
        q.add(('request', c, self))
        await(each(('request', c2, p) in q,
                   has= (c2, p)==(c, self) or (c, self) < (c2, p)) and
              each(p in s, has= some(received(('ack', c2, _p)), has= c2 > c)))
        -- critical_section
        task()
        -- release
        q.remove(('request', c, self))
        send(('release', logical_clock(), self), to= s)

    def receive(msg= ('request', c2, p)):
        q.add(('request', c2, p))
        send(('ack', logical_clock(), self), to= p)

    def receive(msg= ('release', _, p)):
#        q.remove(('request', _, p))  # pattern matching needed for _
#        q.remove(anyof(setof(('request', c, p), ('request', c, _p) in q)))
        for x in setof(('request', c, p), ('request', c, _p) in q):
            q.remove(x)
            break
#        for ('request', c, _p) in q: q.remove('request', c, p); break
#        for (tag, c, p2) in q:
#            if tag == 'request' and p2 == p:
#                q.remove((tag, c, p2)); break

    def run():
        def task():
            output('in cs')
        for i in range(nrequests):
            mutex(task)

        send(('done', self), to= parent())
        await(received(('done',), from_=parent()))
        output('terminating')

In [4]:
nprocs = 2
nrequests = 5

class Node_(da.NodeProcess):

    def __init__(self, procimpl, forwarder, **props):
        super().__init__(procimpl, forwarder, **props)
        self._events.extend([])

    def run(self):
        ps = new(P, num=nprocs)
        print(ps)

In [9]:
from da.common import global_init, initialize_runtime_options

global_init([])

Traceback (most recent call last):
  File "/Users/akshay/Projects/cse535/distalgo_kernel/distalgo_kernel/kernel.py", line 36, in da_execute
    exec_then_eval(code, ctx, ctx)
  File "/Users/akshay/Projects/cse535/distalgo_kernel/distalgo_kernel/kernel.py", line 21, in exec_then_eval
    exp = eval(compile(last, '<string>', mode='eval'), _globals, _locals)
  File "<string>", line 3, in <module>
  File "/Users/akshay/Projects/cse535/distalgo/da/common.py", line 228, in global_init
    sysinit()
  File "/Users/akshay/Projects/cse535/distalgo/da/common.py", line 209, in sysinit
    pid_format = GlobalOptions['pid_format']
KeyError: 'pid_format'

